# Experiment 6

This notebook contains the experiment to investigate the question **Can we control the outbreak using only contact tracing and isolation?**, which is reported on Figure 7 (a) in the [paper](https://arxiv.org/abs/2004.07641).

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [ ]:
import numpy as np
import pickle
import multiprocessing

In [ ]:
from lib.mobilitysim import MobilitySimulator
from lib.parallel import launch_parallel_simulations
from lib.distributions import CovidDistributions
from lib.data import collect_data_from_df
from lib.measures import (MeasureList, Interval,
                          SocialDistancingForSmartTracing,
                          SocialDistancingForPositiveMeasure)

#### Set the random seed for reproducibility

In [ ]:
# Choose random seed
c = 0
# Set it
np.random.seed(c)
# Define prefix string used to save plots
runstr = f'run{c}_'  

#### Set the number of roll-outs to simulate

In [ ]:
random_repeats = 40 # Set to at least 20 to obtain stable results

#### Define the number of parallel workers used for simulations 

In [ ]:
num_workers = multiprocessing.cpu_count() - 1

#### Set the time to simulate

In [ ]:
days_present = 35  # Days since March 10 on April 12 (March 10 is time 0 for our experiments)
days_future = 12 * 7  # Projecting 12 weeks into the future
max_time_future = 24.0 * days_future  # in hours

#### Define population/mobility parameters

Import downsampled version of Tübingen (population downsampled by 20x, sites by 10x) that was used used to generate mobility traces on the fly for inference (sites, home locations, etc)

In [ ]:
case_downsample = 10                       # Downsample of population
mob_settings = 'lib/tu_settings_20_10.pk'  # Mobility settings (should match `case_downsample`)
town_name = 'LK Tübingen'                  # Name of town to fetch case data

# See town-generator.ipynb for an example on how to create the settings
with open(mob_settings, 'rb') as fp:
    obj = pickle.load(fp)
mob = MobilitySimulator(**obj)

#### Load the case data

We adjust the number of cases to the down-scaling of the population

In [ ]:
new_cases_ = collect_data_from_df(town_name, 'new', until=days_present)
resistant_cases_ = collect_data_from_df(town_name, 'recovered', until=days_present)
fatality_cases_ = collect_data_from_df(town_name, 'fatality', until=days_present)

# fatality rate per age group
num_age_groups = fatality_cases_.shape[1] 
fatality_rates_by_age = (fatality_cases_[-1, :] / \
    (new_cases_[-1, :] + fatality_cases_[-1, :] + resistant_cases_[-1, :]))
print('Empirical fatality rates per age group:  ', fatality_rates_by_age.tolist())

# Instantiate correct state transition distributions (estimated from in literature)
distributions = CovidDistributions(fatality_rates_by_age=fatality_rates_by_age)

# Scale down cases based on number of people in simulation
new_cases, resistant_cases, fatality_cases = (
    1/case_downsample * new_cases_, 
    1/case_downsample * resistant_cases_, 
    1/case_downsample * fatality_cases_)
new_cases, resistant_cases, fatality_cases = np.ceil(new_cases), np.ceil(resistant_cases), np.ceil(fatality_cases)

#### Define initial seed count per state 

These numbers are based on the *downscaled* infection counts on April 12.

In [ ]:
with open('summaries/'+f'ave_present_seeds_{c}.pk', 'rb') as logfile:
    present_seeds = pickle.load(logfile)

#### Set epidemic parameters

Inferred using Bayesian optimization.

In [ ]:
inferred_params = {
    'betas': [1.1383] * 5,  # site infectivity by type
}

#### Define standard testing parameters

The parameters are the same used for inference.

In [ ]:
def standard_testing(max_time):
    standard_testing_params = {
        'testing_t_window'    : [0.0, max_time], # in hours
        'testing_frequency'   : 24.0,     # in hours
        'test_reporting_lag'  : 48.0,     # in hours (actual and self-report delay)
        'tests_per_batch'     : 10,       # assume 300 tests/day in LK Tübingen
        'test_smart_delta'    : 24.0 * 3, # in hours
        'test_smart_duration' : 24.0 * 7, # in hours
        'test_smart_action'   : 'isolate', 
        'test_smart_num_contacts'   : 10, 
        'test_targets'        : 'isym',
        'test_queue_policy'   : 'fifo',
        'smart_tracing'       : None, 
    }
    return standard_testing_params

#### Define helper I/O functions to load/save the output of the experiment

In [ ]:
def save_summary(summary, filename):
    with open('summaries/' + filename, 'wb') as fp:
        pickle.dump(summary, fp)
    
def load_summary(filename):
    with open('summaries/' + filename, 'rb') as fp:
        summary = pickle.load(fp)
    return summary

---

## 2. Run the simulations

Define a helper function to run general type of experiment, fixing all parameters above.


In [ ]:
def run(tparam, measure_list, t, local_seeds):

    # add standard measure of positives staying isolated
    measure_list +=  [
        SocialDistancingForPositiveMeasure(
            t_window=Interval(0.0, t), p_stay_home=1.0)
    ]
    measure_list = MeasureList(measure_list)

    # run simulations
    summary = launch_parallel_simulations(
        mob_settings, 
        distributions, 
        random_repeats, num_workers, 
        inferred_params, local_seeds, tparam, measure_list, 
        max_time=t, 
        num_people=mob.num_people, 
        num_sites=mob.num_sites, 
        site_loc=mob.site_loc, 
        home_loc=mob.home_loc, verbose=False)
    return summary
    

Use settings as above and simulate in the future, with additional measures of the same length

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [ ]:
# baseline (no measure for the future starting April 12)
future_baseline = run(standard_testing(max_time_future), [], max_time_future, present_seeds) 

In [ ]:
testing_params_SD_5 = standard_testing(max_time_future)
testing_params_SD_5['test_smart_delta'] = 24.0 * 3 # time window considered for inspecting contacts
testing_params_SD_5['test_smart_action'] = 'isolate'
testing_params_SD_5['test_targets'] = 'isym'

isolation_days = [7] # how many days selected people have to stay in isolation
contacts_isolated = [10, 25] # how many contacts are isolated in the `test_smart_delta` window
duration_weeks_SD_5 = 12 # strategies tested for 12 weeks starting today

summaries_SD_5 = dict()

In [ ]:
for days in isolation_days:
    for contacts in contacts_isolated:
        summaries_SD_5[(days, contacts)] = [future_baseline]


# Isolation strategies
for days in isolation_days:
    for contacts in contacts_isolated:
        for policy in ['basic', 'advanced']:

            testing_params_SD_5['smart_tracing'] = policy
            testing_params_SD_5['test_smart_num_contacts'] = contacts


            # same as tuned plus different isolation strategies for contact tracing
            m = [SocialDistancingForSmartTracing(
                    t_window=Interval(*testing_params_SD_5['testing_t_window']), 
                    p_stay_home=1.0, 
                    test_smart_duration=24.0 * days)
            ]

            res = run(testing_params_SD_5, m, max_time_future, present_seeds) 
            summaries_SD_5[(days, contacts)].append(res)
            
            print(days, contacts, policy, ' done.')
            
save_summary(summaries_SD_5, 'summaries_SD_5.pk')

---

## 3. Plot the results

Import libs

In [ ]:
from lib.plot import Plotter

Reproduce Figure 7 (a) in the paper.

In [ ]:
summaries_SD_5 = load_summary('summaries_SD_5.pk')
plotter = Plotter()

isolation_days = [7]
contacts_isolated = [10, 25]

for i, days in enumerate(isolation_days):
    for j, contacts in enumerate(contacts_isolated):
        summaries_ = summaries_SD_5[(days, contacts)]
        titles_SD_5_ = ['No contact tracing', 'Basic contact tracing', 'Advanced contact tracing']
        plotter.compare_total_infections(
            summaries_, 
            titles=titles_SD_5_, 
            figtitle=(f'Strategies for isolating {contacts} contacts.\n'
                      f'Selected people are isolated for {days} days.'),
            filename=runstr + f'SD_5{i}{j}',
            start_date='2020-04-12',
            figsize=(6,4), acc=500, 
            ymax=2000, errorevery=14)

Not used in the paper

In [ ]:
# plotter = Plotter()

# for i, days in enumerate(isolation_days):
#     for j, contacts in enumerate(contacts_isolated):
#         summaries_ = summaries_SD_5[(days, contacts)]
#         titles_SD_5_ = ['No contact tracing', 'Basic contact tracing', 'Advanced contact tracing']
#         for k, titl in enumerate(titles_SD_5_):
#             plotter.plot_daily_at_home(
#                 summaries_[k],
#                 title='Home isolation for ' + titl, 
#                 filename=runstr + f'SD_6{i}{j}{k}', figsize=(6,4), 
#                 errorevery=14, acc=500)